In [1]:
import os
import torch

# 定义一个函数用于从文件中读取复数并存储到列表中
def read_complex_numbers_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        real_parts = [float(lines[i].strip()) for i in range(0, len(lines), 2)]
        imag_parts = [float(lines[i+1].strip()) for i in range(0, len(lines), 2)]
        complex_numbers = [complex(real, imag) for real, imag in zip(real_parts, imag_parts)]
    return complex_numbers

# 文件夹路径
folder_path = 'data'

# 读取每个文件中的复数并存储到列表
data = []
for i in range(48):
    file_name = f'data{i}.txt'
    file_path = os.path.join(folder_path, file_name)
    if os.path.exists(file_path):
        complex_numbers = read_complex_numbers_from_file(file_path)
        data.append(complex_numbers)

PSS_data = []
for i in range(3):
    file_name = f'PSS{i}.txt'
    file_path = os.path.join(folder_path, file_name)
    if os.path.exists(file_path):
        complex_numbers = read_complex_numbers_from_file(file_path)
        PSS_data.append(complex_numbers)

data_tensor = torch.tensor(data, dtype=torch.complex64)
PSS_data_tensor = torch.tensor(PSS_data, dtype=torch.complex64)

pow_dict = {i: torch.sum(torch.abs(row)) for i, row in enumerate(data_tensor)}
sorted_items = sorted(pow_dict.items(), key=lambda x: x[1], reverse=True)
top_six_indices = [key for key, value in sorted_items[:6]]
print('能量最大的前六个data编号为：', top_six_indices)

def relevant_test(data_index, pss_index):
    max_val = 0
    position = 0
    for i in range(data_tensor.size(1)-PSS_data_tensor.size(1)):
        ret = torch.sum(data_tensor[data_index, i:i+PSS_data_tensor.size(1)] * torch.conj(PSS_data_tensor[pss_index])).abs()
        if ret > max_val:
            position = i
            max_val = ret
    return max_val, position

result = {}
for pss_index in range(PSS_data_tensor.size(0)):
    print(f'PSS{pss_index}:')
    for data_index in top_six_indices:
        max_val, position = relevant_test(data_index, pss_index)
        print(f'data{data_index}:', max_val.item(), 'position:', position)
        result[f'PSS{pss_index}-data{data_index}'] = [max_val.item(), position]

max_values = {}
for key, value in result.items():
    pss, data_num = key.split('-')
    if pss not in max_values or value[0] > max_values[pss][0]:
        max_values[pss] = (value[0], data_num)

for pss, (max_val, data_num) in max_values.items():
    print(f"{pss} corresponds to {data_num}")


能量最大的前六个data编号为： [18, 6, 12, 0, 19, 7]
PSS0:
data18: 2736769.5 position: 200
data6: 2587698.25 position: 3649
data12: 2211295.5 position: 2384
data0: 2354982.25 position: 2499
data19: 2896821.5 position: 7666
data7: 4404822.0 position: 7666
PSS1:
data18: 2944468.75 position: 5112
data6: 2791073.75 position: 12396
data12: 2210388.75 position: 1830
data0: 2689116.75 position: 8625
data19: 2018986.5 position: 8716
data7: 5189659.0 position: 12749
PSS2:
data18: 2642151.25 position: 1478
data6: 2353013.5 position: 12089
data12: 2011171.625 position: 737
data0: 2449533.5 position: 8586
data19: 2074767.375 position: 7771
data7: 3909197.75 position: 7328
PSS0 corresponds to data7
PSS1 corresponds to data7
PSS2 corresponds to data7
